In [1]:
!lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          2
On-line CPU(s) list:             0,1
Thread(s) per core:              2
Core(s) per socket:              1
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           85
Model name:                      Intel(R) Xeon(R) CPU @ 2.00GHz
Stepping:                        3
CPU MHz:                         2000.178
BogoMIPS:                        4000.35
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       32 KiB
L1i cache:                       32 KiB
L2 cache:                        1 MiB
L3 cache:                        38.5 MiB
NUMA node0 CPU(s):               0,1
Vulnerability 

In [2]:
!nvidia-smi


Thu Dec 29 06:21:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!python -V
!cmake --version

Python 3.7.12
cmake version 3.22.5

CMake suite maintained and supported by Kitware (kitware.com/cmake).


In [95]:
!pip install -q pyvi
!pip install -q visen
!pip install -q emoji
!pip install -q pretty-confusion-matrix
!pip install -q tf2onnx
!pip install -q onnx
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 11.3 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.0 MB/s eta 0:00:00


In [192]:
!pip install --upgrade scikit-learn

In [194]:
import re
from unicodedata import normalize
from string import punctuation
import visen
import emoji

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, log_loss
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from pretty_confusion_matrix import pp_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer


In [195]:
import sklearn
sklearn.__version__

'1.0.2'

In [72]:
opening_ls = ['[', '{', '⁅', '〈', '⎡', '⎢', '⎣', '⎧', '⎨', '⎩', '❬', '❰', '❲', '❴', '⟦', '⟨', '⟪', '⟬', '⦃', '⦇', '⦉',
              '⦋', '⦍', '⦏', '⦑', '⦓', '⦕', '⦗', '⧼', '⸂', '⸄', '⸉', '⸌', '⸜', '⸢', '⸤', '⸦', '〈', '《', '「', '『',
              '【', '〔', '〖', '〘', '〚', '﹛', '﹝', '［', '｛', '｢', '｣']

closing_ls = [']', '}', '⁆', '〉', '⎤', '⎥', '⎦', '⎫', '⎬', '⎭', '❭', '❱', '❳', '❵', '⟧', '⟩', '⟫', '⟭', '⦄', '⦈', '⦊',
              '⦌', '⦎', '⦐', '⦒', '⦔', '⦖', '⦘', '⧽', '⸃', '⸅', '⸊', '⸍', '⸝', '⸣', '⸥', '⸧', '〉', '》', '」', '』',
              '】', '〕', '〗', '〙', '〛', '﹜', '﹞', '］', '｝', '｣']

opening_brackets = {key: '(' for key in opening_ls}
closing_brackets = {key: ')' for key in closing_ls}

### constant
PUNC = '!\"#$&()*+,-–−./:;=?@[\]^_`{|}~”“`°²ˈ‐ㄧ‛∼’'  # remove <> for number_sym and unknown_sym
NUMBER_SYM = '<NUMBER>'
UNKNOWN_SYM = ' <UNKNUM> '  # mix number with character, ex: 6.23A
VIETNAMESE_CHARS = 'aàảãáạăằẳẵắặâầẩẫấậbcdđeèẻẽéẹêềểễếệfghiìỉĩíịjklmnrstoòỏõóọôồổỗốộơờởỡớợpquùủũúụưừửữứựvwxyỳỷỹýỵz'
re_email = '([a-zA-Z0-9._%+-]+@[a-zA-Z0-9-]+\\.[a-zA-Z]{2,4}(\\.?[a-zA-Z]{2,4})?)'
re_url = '(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})'
re_url2 = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
re_image = '(tập[\s_]tin|hình|file|image|imagesize).*?(jpg|svg|png|gif|jpeg|ogg|tif|width)'
re_num_and_decimal = '[0-9]*[,.\-]*[0-9]*[,.\-]*[0-9]*[.,\-]*[0-9]*[,.\-]*[0-9]+[.,]?'
re_unknown = '[a-z]+[\d]+[\w]*|[\d]+[a-z]+[\w]*'
re_vnese_txt = r'[^a-z0-9A-ZàáãạảăắằẳẵặâấầẩẫậèéẹẻẽêềếểễệđìíĩỉịòóõọỏôốồổỗộơớờởỡợùúũụủưứừửữựỳỵỷỹýÀÁÃẠẢĂẮẰẲẴẶÂẤẦẨẪẬÈÉẸẺẼÊỀẾỂỄỆĐÌÍĨỈỊÒÓÕỌỎÔỐỒỔỖỘƠỚỜỞỠỢÙÚŨỤỦƯỨỪỬỮỰỲỴỶỸÝ\s|_]'
re_truncate_unknown = '(<UNKNUM>\s*)+'


special_punc = {'”': '"', '': '', "’": "'", "`": "'"}


def replace_all(replacer: dict, txt: str) -> str:
    for old, new in replacer.items():
        txt = txt.replace(old, new)
    return txt


def replace_num(text: str) -> str:
    text = re.sub(re_num_and_decimal, UNKNOWN_SYM, text)
    return text


def replace_unknown(text: str) -> str:
    text = re.sub(re_unknown, UNKNOWN_SYM, text)
    return text


def unicode_normalizer(text, forms: list = ['NFKC', 'NKFD', 'NFC', 'NFD']) -> str:
    for form in forms:
        text = normalize(form, text)
    return text


def normalize_bracket(text: str) -> str:
    text = replace_all(opening_brackets, text)
    text = replace_all(closing_brackets, text)
    text = re.sub(r"[\(\[].*?[\)\]]", " ", text)
    return text


def remove_punc(text: str) -> str:
    r = re.compile(r'[\s{}]+'.format(re.escape(PUNC)))
    text = r.split(text)
    return ' '.join(i for i in text if i)


def truncate_unknown(text: str) -> str:
    text = re.sub(re_truncate_unknown, UNKNOWN_SYM, text)
    return text


def norm(text: str) -> str:
    text = str(text)
    text = text.lower()
    text = text.split('\n')[0]
    text = unicode_normalizer(text, ["NFKC", "NFKD", "NFD", "NFC"])
    text = replace_all(special_punc, text)
    text = normalize_bracket(text)
    text = replace_unknown(text)
    text = replace_num(text)
    text = remove_punc(text)
    text = truncate_unknown(text)
    text = re.sub(re_vnese_txt, " ", text)
    text = text.strip()
    return visen.clean_tone(text)

sample_txts = [
    '〖𝔰𝔬𝔪𝔢 𝔣𝔞𝔫𝔠𝔶 𝔭𝔯𝔬𝔡𝔲𝔠𝔱 𝔫𝔞𝔪𝔢〗',
    '𝖘𝖔𝖒𝖊 𝖋𝖆𝖓𝖈𝖞 𝖕𝖗𝖔𝖉𝖚𝖈𝖙 𝖓𝖆𝖒𝖊',
    '❰some product code which will overfit this sample❱ 𝓼𝓸𝓶𝓮 𝓯𝓪𝓷𝓬𝔂 𝓹𝓻𝓸𝓭𝓾𝓬𝓽 𝓷𝓪𝓶𝓮',
    '𝓈𝑜𝓂𝑒 𝒻𝒶𝓃𝒸𝓎 𝓅𝓇𝑜𝒹𝓊𝒸𝓉 𝓃𝒶𝓂𝑒',
    '{𝕤𝕠𝕞𝕖 𝕗𝕒𝕟𝕔𝕪 𝕡𝕣𝕠𝕕𝕦𝕔𝕥 𝕟𝕒𝕞𝕖',
    '☯😝ｓｏｍｅ ｆａｎｃｙ ｐｒｏｄｕｃｔ ｎａｍｅ☯😝',
    '𝐬𝐨𝐦𝐞 𝐟𝐚𝐧𝐜𝐲 𝐩𝐫𝐨𝐝𝐮𝐜𝐭 𝐧𝐚𝐦𝐞',
    '𝘀𝗼𝗺𝗲 𝗳𝗮𝗻𝗰𝘆 𝗽𝗿𝗼𝗱𝘂𝗰𝘁 𝗻𝗮𝗺𝗲',
    '𝘴𝘰𝘮𝘦 𝘧𝘢𝘯𝘤𝘺 𝘱𝘳𝘰𝘥𝘶𝘤𝘵 𝘯𝘢𝘮𝘦',
    '▄▀▄▀▄▀ 𝙨𝙤𝙢𝙚 𝙛𝙖𝙣𝙘𝙮 𝙥𝙧𝙤𝙙𝙪𝙘𝙩 𝙣𝙖𝙢𝙚 ▄▀▄▀▄▀',
    '🌌  🎀 𝚜𝚘𝚖𝚎 𝚏𝚊𝚗𝚌𝚢 𝚙𝚛𝚘𝚍𝚞𝚌𝚝 𝚗𝚊𝚖𝚎🌌  🎀 ',
    '(っ◔◡◔)っ ♥ some fancy product name ♥',
]

for sample in sample_txts:
    print(f'sample \"{sample}\", after normalize: \"{mask(sample)}\"')


sample "〖𝔰𝔬𝔪𝔢 𝔣𝔞𝔫𝔠𝔶 𝔭𝔯𝔬𝔡𝔲𝔠𝔱 𝔫𝔞𝔪𝔢〗", after normalize: ""
sample "𝖘𝖔𝖒𝖊 𝖋𝖆𝖓𝖈𝖞 𝖕𝖗𝖔𝖉𝖚𝖈𝖙 𝖓𝖆𝖒𝖊", after normalize: "some fancy product name"
sample "❰some product code which will overfit this sample❱ 𝓼𝓸𝓶𝓮 𝓯𝓪𝓷𝓬𝔂 𝓹𝓻𝓸𝓭𝓾𝓬𝓽 𝓷𝓪𝓶𝓮", after normalize: "some fancy product name"
sample "𝓈𝑜𝓂𝑒 𝒻𝒶𝓃𝒸𝓎 𝓅𝓇𝑜𝒹𝓊𝒸𝓉 𝓃𝒶𝓂𝑒", after normalize: "some fancy product name"
sample "{𝕤𝕠𝕞𝕖 𝕗𝕒𝕟𝕔𝕪 𝕡𝕣𝕠𝕕𝕦𝕔𝕥 𝕟𝕒𝕞𝕖", after normalize: "some fancy product name"
sample "☯😝ｓｏｍｅ ｆａｎｃｙ ｐｒｏｄｕｃｔ ｎａｍｅ☯😝", after normalize: "some fancy product name"
sample "𝐬𝐨𝐦𝐞 𝐟𝐚𝐧𝐜𝐲 𝐩𝐫𝐨𝐝𝐮𝐜𝐭 𝐧𝐚𝐦𝐞", after normalize: "some fancy product name"
sample "𝘀𝗼𝗺𝗲 𝗳𝗮𝗻𝗰𝘆 𝗽𝗿𝗼𝗱𝘂𝗰𝘁 𝗻𝗮𝗺𝗲", after normalize: "some fancy product name"
sample "𝘴𝘰𝘮𝘦 𝘧𝘢𝘯𝘤𝘺 𝘱𝘳𝘰𝘥𝘶𝘤𝘵 𝘯𝘢𝘮𝘦", after normalize: "some fancy product name"
sample "▄▀▄▀▄▀ 𝙨𝙤𝙢𝙚 𝙛𝙖𝙣𝙘𝙮 𝙥𝙧𝙤𝙙𝙪𝙘𝙩 𝙣𝙖𝙢𝙚 ▄▀▄▀▄▀", after normalize: "some fancy product name"
sample "🌌  🎀 𝚜𝚘𝚖𝚎 𝚏𝚊𝚗𝚌𝚢 𝚙𝚛𝚘𝚍𝚞𝚌𝚝 𝚗𝚊𝚖𝚎🌌  🎀 ", after normalize: "some fancy product name"
sample "(っ◔◡◔)っ ♥ some fancy product name ♥", after normalize: "some fancy product name"


In [7]:
stopwords = pd.read_csv('/kaggle/input/data-dl-final/custom_stop_words.csv', header=0, encoding='utf-8')
stopwords = stopwords['words'].tolist()
masked_data = '/kaggle/input/data-dl-final/masked_data.csv'
masked = pd.read_csv(masked_data, encoding='utf8')
non_masked = pd.read_csv('/kaggle/input/data-dl-final/shopee_data.csv', header = 0, encoding='utf8', sep='\t', on_bad_lines='skip')

In [8]:
print(masked.shape)
print(non_masked.shape)
masked

(1138950, 2)
(1138950, 2)


,sample,label
0,miếng dán bàn phím không dây logitech <UNKNUM>,Điện tử - Điện máy
1,nắp tròn gắn bàn phím bluetooth cho điện thoại...,Điện tử - Điện máy
2,pin laptop dell latitude <UNKNUM>,Điện tử - Điện máy
3,pin hp envy <UNKNUM> envy <UNKNUM> envy <UNKNU...,Điện tử - Điện máy
4,đầu nối chuyển đổi dc <UNKNUM> cho laptop,Điện tử - Điện máy
...,...,...
1138945,váy body lưới dài đen m,Thời trang
1138946,đầm hoa nổi tay bồng cổ u s,Thời trang
1138947,bộ dior chữ thêu trước ngực trắng l,Thời trang
1138948,dép ny sọc đen trắng <UNKNUM>,Thời trang


In [9]:
masked['sample'] = masked['sample'].str.replace('<UNKNUM>','')
masked['sample'] = masked['sample'].str.replace(r'\s+', ' ', regex=True)
masked

,sample,label
0,miếng dán bàn phím không dây logitech,Điện tử - Điện máy
1,nắp tròn gắn bàn phím bluetooth cho điện thoại...,Điện tử - Điện máy
2,pin laptop dell latitude,Điện tử - Điện máy
3,pin hp envy envy envy envy,Điện tử - Điện máy
4,đầu nối chuyển đổi dc cho laptop,Điện tử - Điện máy
...,...,...
1138945,váy body lưới dài đen m,Thời trang
1138946,đầm hoa nổi tay bồng cổ u s,Thời trang
1138947,bộ dior chữ thêu trước ngực trắng l,Thời trang
1138948,dép ny sọc đen trắng,Thời trang


In [10]:
masked['label'].value_counts()

unknown               314845
Điện tử - Điện máy    260100
Thời trang            196910
Mỹ phẩm               189308
Mẹ & Bé               177787
Name: label, dtype: int64

In [11]:
masked = masked[~(masked['label'] == 'unknown')]
masked['label'].value_counts()

Điện tử - Điện máy    260100
Thời trang            196910
Mỹ phẩm               189308
Mẹ & Bé               177787
Name: label, dtype: int64

In [12]:
masked.dropna(inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [13]:
X= masked["sample"]

In [14]:
y = masked['label']

# **Tfidf**

In [84]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import keras
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
import tensorflow as tf
import keras
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

In [16]:
le = LabelEncoder()
y = le.fit_transform(y)
y = np_utils.to_categorical(y)
y

array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [18]:
y_train

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)

In [19]:
tfidf_vect = TfidfVectorizer()
# tfidf_vect.fit(X_train)#.ravel()) # learn vocabulary and idf from training set
X_train =  tfidf_vect.fit_transform(X_train)#.ravel())
# assume that we don't have test set before
X_test =  tfidf_vect.transform(X_test)

In [20]:
X_train.shape

(576809, 67332)

In [21]:
X_train[0].shape[1]

67332

In [22]:
# Dimensionality reduction


In [23]:
svd = TruncatedSVD(n_components=400, random_state=42)
X_train = svd.fit_transform(X_train)
X_test=svd.transform(X_test)
print(f"Total variance explained: {np.sum(svd.explained_variance_ratio_):.2f}")
print(X_train.shape)
print(y_train)

Total variance explained: 0.48
(576809, 400)
[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [24]:
model = Sequential()
model.add(Flatten(input_shape=(400,)))
model.add(Dense(256, input_dim=200, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(160, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(120, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(80, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
es = keras.callbacks.EarlyStopping(monitor='val_f1_m', min_delta=0, patience=3, verbose=0, mode='auto', baseline=None, restore_best_weights=False)

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 400)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               102656    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               51400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 160)               32160     
_________________________________________________________________
dropout_2 (Dropout)          (None, 160)               0

2022-12-29 06:25:27.645611: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-29 06:25:27.649958: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-29 06:25:27.650900: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-29 06:25:27.652062: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [25]:
y_train

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)

In [26]:
model.fit(X_train, y_train, batch_size = 1024, epochs = 20, callbacks=[es])

2022-12-29 06:25:30.902864: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 922894400 exceeds 10% of free system memory.
2022-12-29 06:25:31.957624: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 922894400 exceeds 10% of free system memory.
2022-12-29 06:25:32.721342: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20
564/564 [==============================] - 4s 3ms/step - loss: 0.1799 - accuracy: 0.9313
Epoch 2/20
564/564 [==============================] - 2s 4ms/step - loss: 0.0726 - accuracy: 0.9774
Epoch 3/20
564/564 [==============================] - 2s 4ms/step - loss: 0.0622 - accuracy: 0.9804
Epoch 4/20
564/564 [==============================] - 2s 3ms/step - loss: 0.0567 - accuracy: 0.9821
Epoch 5/20
564/564 [==============================] - 2s 4ms/step - loss: 0.0524 - accuracy: 0.9832
Epoch 6/20
564/564 [==============================] - 2s 3ms/step - loss: 0.0488 - accuracy: 0.9844
Epoch 7/20
564/564 [==============================] - 2s 3ms/step - loss: 0.0475 - accuracy: 0.9847
Epoch 8/20
564/564 [==============================] - 2s 4ms/step - loss: 0.0444 - accuracy: 0.9856
Epoch 9/20
564/564 [==============================] - 2s 4ms/step - loss: 0.0431 - accuracy: 0.9860
Epoch 10/20
564/564 [==============================] - 2s 3ms/step - loss: 0.0417 - accuracy: 0.9864

In [27]:
pred = model.predict(X_test)

In [28]:
pred

array([[1.4221607e-10, 1.0000000e+00, 9.0081583e-13, 1.3057433e-09],
       [1.4977683e-08, 1.0000000e+00, 5.6461953e-11, 1.5583054e-09],
       [1.1460472e-10, 8.0769023e-11, 8.5183645e-13, 1.0000000e+00],
       ...,
       [3.6366878e-14, 1.5073444e-14, 1.2494369e-18, 1.0000000e+00],
       [1.5436259e-14, 9.6720771e-19, 1.0000000e+00, 5.6087325e-27],
       [2.2827176e-12, 6.7503544e-16, 1.0000000e+00, 1.2138475e-21]],
      dtype=float32)

In [29]:
pred = np.argmax(pred, axis=1)
pred

array([1, 1, 3, ..., 3, 2, 2])

In [30]:
y_test = np.argmax(y_test, axis=1)
y_test

array([1, 1, 3, ..., 3, 2, 2])

In [31]:
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98     53481
           1       0.98      0.98      0.98     56708
           2       0.99      0.99      0.99     58996
           3       0.99      0.99      0.99     78020

    accuracy                           0.99    247205
   macro avg       0.98      0.98      0.98    247205
weighted avg       0.99      0.99      0.99    247205



In [80]:
def predict(batch):
    batch = svd.transform(tfidf_vect.transform(norm(i) for i in batch))
    return le.inverse_transform(np.argmax(model.predict(batch), axis=1))
predict(['iphone', 'ao dai 123'])

(2, 400)


array(['Điện tử - Điện máy', 'Thời trang'], dtype=object)

In [77]:
import tf2onnx
import onnx

In [139]:
sample = ['macbook pro']
input_signature = [tf.TensorSpec((1, 400),tf.float64, name="input")]
dynamic_axes={'text_embedding' : {0 : 'batch_size'}, 'log_softmax' : {}}
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature, opset=13)
onnx.save(onnx_model, "keras_classifier.onnx")

2022-12-29 08:21:22.135561: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-29 08:21:22.135975: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-12-29 08:21:22.136209: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-12-29 08:21:22.136707: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-29 08:21:22.137126: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-29 08:21:22.137509: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] su

In [140]:
import onnxruntime as ort

In [141]:
so = ort.SessionOptions()
so.add_session_config_entry('session.load_model_format', 'ONNX')
so.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
so.optimized_model_filepath = '/kaggle/working/keras_classifier.onnx'
ort_sess = ort.InferenceSession('/kaggle/working/keras_classifier.onnx', providers=['CPUExecutionProvider'], sess_options=so)

2022-12-29 08:21:26.149740806 [W:onnxruntime:, inference_session.cc:1458 Initialize] Serializing optimized model with Graph Optimization level greater than ORT_ENABLE_EXTENDED and the NchwcTransformer enabled. The generated model may contain hardware specific optimizations, and should only be used in the same environment the model was optimized in.


In [115]:
print(ort_sess.get_inputs()[0].name)
print(ort_sess.get_outputs()[0].name)

input
dense_5


In [165]:
def predict_with_onnx(batch):
    batch = svd.transform(tfidf_vect.transform(norm(i) for i in batch))
    print(batch.shape)
    print(type(batch))
    pred = ort_sess.run(None, {'input': batch})
    pred = np.argmax(pred[0], axis=1)
    return le.inverse_transform([pred])

In [166]:
predict_with_onnx(['ao choang'])

(1, 400)
<class 'numpy.ndarray'>


array(['Thời trang'], dtype=object)

In [145]:
import pickle
import lzma
import gzip
from sklearn.pipeline import Pipeline

In [150]:
pipe = Pipeline([('vectorize', tfidf_vect), ('trucate', svd)])

In [172]:
trf = pipe.transform(['ao choang'])
pred = ort_sess.run(None, {'input': trf})
pred = np.argmax(pred[0], axis=1)
print(le.inverse_transform([pred]))

(1, 400)
<class 'numpy.ndarray'>
['Thời trang']


In [186]:
def predict_with_onnx_and_skpipeline(sample):
    sample = pipe.transform([norm(sample)])
    pred = ort_sess.run(None, {'input': sample})
    pred = np.argmax(pred[0], axis=1)
    return le.inverse_transform([pred])

In [187]:
predict_with_onnx_and_skpipeline('ao choang')

array(['Thời trang'], dtype=object)

In [188]:
with open('Vectorizer.pkl', 'wb') as f:
    pickle.dump(pipe, f)